# lustre

In [ ]:
#!import ../../../polyglot/lib/fsharp/Notebooks.dib

## lustre

### attribute

In [ ]:
nominal attribute t =
    `(
        backend_switch `(()) `({}) {
            Gleam = (fun () => global "import lustre/internals/vdom") : () -> ()
        }
        $'' : $'vdom.Attribute(`t)'
    )

### element

In [ ]:
nominal element t =
    `(
        backend_switch `(()) `({}) {
            Gleam = (fun () => global "import lustre/element.{type Element}") : () -> ()
        }
        $'' : $'Element(`t)'
    )

### text

In [ ]:
inl text forall t. (text : string) : element t =
    global "import lustre/element/html"
    $'element.text(!text)'

### on_click

In [ ]:
inl on_click forall t. (msg : t) : attribute t =
    global "import lustre/event"
    $'event.on_click(!msg)'

### style

In [ ]:
inl style forall t. (styles : list (string * string)) : attribute t =
    global "import lustre/attribute"
    inl styles = styles |> listm'.box
    $'attribute.style(!styles)'

### div

In [ ]:
inl div forall t. (attributes : list (attribute t)) (children : list (element t)) : element t =
    global "import lustre/element/html"
    inl attributes = attributes |> listm'.box
    inl children = children |> listm'.box
    $'html.div(!attributes, !children)'

### p

In [ ]:
inl p forall t. (attributes : list (attribute t)) (children : list (element t)) : element t =
    global "import lustre/element/html"
    inl attributes = attributes |> listm'.box
    inl children = children |> listm'.box
    $'html.p(!attributes, !children)'

### button

In [ ]:
inl button forall t. (attributes : list (attribute t)) (children : list (element t)) : element t =
    global "import lustre/ui/button.{button}"
    inl attributes = attributes |> listm'.box
    inl children = children |> listm'.box
    $'button(!attributes, !children)'

### centre

In [ ]:
inl centre forall t. (attributes : list (attribute t)) (children : element t) : element t =
    global "import lustre/ui/centre.{centre}"
    inl attributes = attributes |> listm'.box
    $'centre(!attributes, !children)'

### app

In [ ]:
nominal app model msg = $'lustre.App(`model, `model, `msg)'

### simple

In [ ]:
inl simple forall model msg.
    (init : model -> model)
    (update : model * msg -> model)
    (view : model -> element msg)
    : app model msg
    =
    global "import lustre"
    $'lustre.simple(!init, fn (a, b) { !update(#(a, b)) }, !view)'

### start

In [ ]:
inl start app =
    $'let assert Ok(_) = lustre.start(!app, "#app", 0)' : ()

### model

In [ ]:
type model = int

### init

In [ ]:
inl init initial_count =
    if initial_count < 0
    then 0
    else initial_count

### msg

In [ ]:
union msg =
    | Incr
    | Decr

### update

In [ ]:
inl update (model, msg) =
    match msg with
    | Incr => model + 1
    | Decr => model - 1

### view

In [ ]:
inl view model =
    div [
        style [
            "display", "flex"
        ]
    ] [
        div [
            style [
                "display", "flex"
                "flex-direction", "column"
            ]
        ] [
            button [
                on_click Incr
            ] [
                text "+"
            ]
            p [
                style [
                    "text-align", "center"
                ]
            ] [
                model |> sm'.obj_to_string |> text
            ]
            button [
                on_click Decr
            ] [
                text "-"
            ]
        ]
    ]

### main'

In [ ]:
inl main' () =
    inl app : _ model _ = simple init update view
    app |> start

In [ ]:
//// test
///! gleam -t javascript -d lustre=\">=4.6.0\" lustre_ui=\"1.0.0-rc.1\" gleam_json=\">=2.0.0\"

main' ()

<!doctype html>